In [ ]:
#import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
from neural_network import NeuralNetwork
from test_utils import generate_test_dataset, train_test, accuracy

# Exercise 3

### Question A

In [ ]:
x = np.array([[0, 0],
              [0, 1],
              [1, 0],
              [1, 1]])

y = np.array([[0],
              [1],
              [1],
              [0]])

In [ ]:
df = pd.DataFrame(np.concatenate((x, y), axis=1), columns=('x1', 'x2', 'y'))
df.style.hide()

In [ ]:
plt.figure(figsize=(2,2))

plt.plot(*x[0], 'o', color='blue')
plt.plot(*x[1], 'o', color='red')
plt.plot(*x[2], 'o', color='red')
plt.plot(*x[3], 'o', color='blue')
plt.grid()

plt.show()

In [ ]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size, output_size, one_hot=False, activation_function='reLU'):
        # Initialize number of nodes of input, hidden, and output layer
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.accuracies = [] # Initialize accuracies


        # Initialize weights (w) and biases (b)
        # input -> hidden
        self.w1 = np.random.rand(self.input_size, self.hidden_size) # 35*10
        self.b1 = np.zeros((1, self.hidden_size)) # 1*10

        # hidden -> output
        self.w2 = np.random.rand(self.hidden_size, self.output_size) # 10*1
        self.b2 = np.zeros((1, self.output_size)) # 1*1

        #adam parameters
        self.beta1 = 0.9
        self.beta2 = 0.999
        self.epsilon = 1e-8

        # Increment the time step
        self.t =1
        
        # Activation function
        self.activation_function = activation_function
        
        # Use of one_hot encoding
        self.one_hot = one_hot
        
       # Initialize moving average variables for weights
        self.m_w1 = np.zeros_like(self.w1)
        self.v_w1 = np.zeros_like(self.w1)
        self.m_w2 = np.zeros_like(self.w2)
        self.v_w2 = np.zeros_like(self.w2)

        # Initialize moving average variables for biases
        self.m_b1 = np.zeros_like(self.b1)
        self.v_b1 = np.zeros_like(self.b1)
        self.m_b2 = np.zeros_like(self.b2)
        self.v_b2 = np.zeros_like(self.b2)

        # Record data
        self.losses = []
    
   
    def activation(self, x, activation_function, deriv=False):
        """Activation function (sigmoid, reLU, softmax)"""
        if activation_function == 'sigmoid':
            if deriv:
                return self.activation(x, activation_function) * (1 - self.activation(x, activation_function))
            return 1 / (1 + np.exp(-x))
        elif activation_function == 'reLU':
            if deriv:
                return np.where(x > 0, 1, 0)  # Derivative of ReLU
            return np.maximum(0, x)



    def one_hot_encoding(self, y):
        """Encode digit to one-hot (used in error calculation of backpropagation)"""
        y = y.reshape(-1, 10)        
        encoded_y = np.zeros((y.size, y.max() + 1))
        encoded_y[np.arange(y.size), y.flatten()] = 1
        encoded_y = encoded_y.T
        return encoded_y



    def feedforward(self, x):
        """Feedforward"""
        # input -> hidden
        self.z1 = np.dot(x, self.w1) + self.b1
        self.a1 = self.activation(self.z1, self.activation_function)

        # hidden -> output
        self.z2 = np.dot(self.a1, self.w2) + self.b2
        self.a2 = self.activation(self.z2, self.activation_function)

        return self.a2


    def backpropagation(self, x, y, learning_rate):
        """Backpropagation"""        
        self.m = y.size
        
        self.d_z2 = self.a2 - y # 10*1
        self.d_w2 = np.dot(self.a1.T, self.d_z2) / self.m # 10*10 @ 10*1 = 10*1
        self.d_b2 = np.sum(self.d_z2, axis=0, keepdims=True) / self.m # 1*1
        
        self.d_z1 = np.dot(self.d_z2, self.w2.T) * self.activation(self.z1, self.activation_function, deriv=True) # 10*1 @ 1*10
        self.d_w1 = np.dot(x.T, self.d_z1) / self.m
        self.d_b1 = np.sum(self.d_z1, axis=0, keepdims=True) / self.m

 
    
    def train(self, x, y, learning_rate, epochs):
        """Optimize weight and bias parameters for adam""" 
        self.input = x
        self.output = y
        self.learning_rate = learning_rate
        self.epochs = epochs

        if self.one_hot:
            y = self.one_hot_encoding(y)

        for epoch in range(epochs):
            self.feedforward(x)
            self.backpropagation(x, y, learning_rate)

            # Calculate accuracy
            #predictions = (self.predict(x) > 0.5).astype(int)  # Convert to integers
            predictions = np.round(self.predict(x))  
            accuracy = np.mean(predictions == y)
            self.accuracies.append(accuracy)  # Store accuracy
            
            if not epoch % (epochs / 10):
                loss = self.cross_entropy_loss(y, self.a2)
                self.losses.append(loss)
                print(f'Epoch {epoch}: {loss}')

            self.update_adam()

            
    def update_adam(self):
        # Update variables
        self.m_w1 = self.beta1 * self.m_w1 + (1 - self.beta1) * self.d_w1
        self.v_w1 = self.beta2 * self.v_w1 + (1 - self.beta2) * (self.d_w1 ** 2)
        self.m_b1 = self.beta1 * self.m_b1 + (1 - self.beta1) * self.d_b1
        self.v_b1 = self.beta2 * self.v_b1 + (1 - self.beta2) * (self.d_b1 ** 2)
        self.m_w2 = self.beta1 * self.m_w2 + (1 - self.beta1) * self.d_w2
        self.v_w2 = self.beta2 * self.v_w2 + (1 - self.beta2) * (self.d_w2 ** 2)
        self.m_b2 = self.beta1 * self.m_b2 + (1 - self.beta1) * self.d_b2
        self.v_b2 = self.beta2 * self.v_b2 + (1 - self.beta2) * (self.d_b2 ** 2)
    

        self.t += 1 
        
        # Bias correction for m and v variables
        self.m_w1_hat = self.m_w1 / (1 - self.beta1 ** self.t)
        self.m_w2_hat = self.m_w2 / (1 - self.beta1 ** self.t)
        self.m_b1_hat = self.m_b1 / (1 - self.beta1 ** self.t)
        self.m_b2_hat = self.m_b2 / (1 - self.beta1 ** self.t)
        self.v_w1_hat = self.v_w1 / (1 - self.beta2 ** self.t)
        self.v_w2_hat = self.v_w2 / (1 - self.beta2 ** self.t)
        self.v_b1_hat = self.v_b1 / (1 - self.beta2 ** self.t)
        self.v_b2_hat = self.v_b2 / (1 - self.beta2 ** self.t)

    
        # Update weights and biases with bias correction
        self.w1 -= self.m_w1_hat * learning_rate / (np.sqrt(self.v_w1_hat) + self.epsilon)
        self.w2 -= self.m_w2_hat * learning_rate / (np.sqrt(self.v_w2_hat) + self.epsilon)
        self.b1 -= self.m_b1_hat * learning_rate / (np.sqrt(self.v_b1_hat) + self.epsilon)
        self.b2 -= self.m_b2_hat * learning_rate / (np.sqrt(self.v_b2_hat) + self.epsilon)


    def cross_entropy_loss(self, y_true, y_pred):
        """Calculate categorical cross-entropy loss"""
        epsilon = 1e-10  # Small value to prevent log(0) cases
        y_pred_clipped = np.clip(y_pred, epsilon, 1 - epsilon)  # Clip predictions to avoid log(0)
        m = y_true.shape[0]  # Number of examples
        loss = -np.sum(y_true * np.log(y_pred_clipped)) / m
        return loss

    def predict(self, x):
        """Get feedforward result with current parameters"""
        return self.feedforward(x)
        
    def graph(self, graph_type, name=None, xlim=None):
        """Graph information about neural network performance"""
        
        # Check graph type
        if graph_type is None:
            raise ValueError('Unknown graph type')
        
        # Generate name if no name is given
        if name is None:
            name = datetime.now().strftime('%Y_%m_%d_%H_%M_%S_%f')
    
        # Add parameter box for plot
        parameters = (
            'Parameters\n\n'
            f'Input nodes: {self.input_size}\n'
            f'Hidden nodes: {self.hidden_size}\n'
            f'Output nodes: {self.output_size}\n'
            f'Learning rate: {self.learning_rate}\n'
            f'Activation function: {self.activation_function}\n'
            f'Epochs: {self.epochs}'
        )
    
        fig, (ax1, ax2) = plt.subplots(1, 2, gridspec_kw={'width_ratios': [8, 1]}, figsize=(7.2, 4.8))
        ax2.text(-0.2, 0.7, parameters, bbox=dict(facecolor='white', alpha=0.5), transform=ax2.transAxes)
        ax2.axis('off')
    
        # Set x-axis limits
        if xlim is not None:
            ax1.set_xlim(xlim)
    
        # Plot loss vs epoch
        if graph_type == 'loss_epoch':
            ax1.plot(self.losses)            
            ax1.set_title('Loss vs epoch')
            ax1.set_xlabel('Epoch')
            ax1.set_ylabel('Loss')
            ax1.grid()
            fig.savefig(f'./graphs/loss_{name}', dpi=300)
    
        # Plot accuracy vs epoch 
        if graph_type == 'accuracy_epoch':
            ax1.plot(self.accuracies) 
            ax1.set_title('Accuracy vs Epoch') 
            ax1.set_xlabel('Epoch') 
            ax1.set_ylabel('Accuracy')
            ax1.grid() 
            fig.savefig(f'./graphs/accuracy_{name}', dpi=300)
    
        # Plot confusion matrix
        elif graph_type == 'confusion':
            matrix = np.array([self.feedforward(value).squeeze() for value in self.input]).T
            normalized_matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]  # Normalize confusion matrix
            im = ax1.matshow(normalized_matrix, cmap='viridis')

    
    
            for i in range(matrix.shape[0]):                
                for j in range(matrix.shape[1]):
                    ax1.text(j, i, f'{np.round(matrix[i, j], 2)}', ha='center', va='center', color='white', size='8')
    
            ax1.set_title('Confusion matrix')
            ax1.set_xlabel('Predicted digit')
            ax1.set_ylabel('Real digit')
            ax1.set_xticks(np.arange(len(self.input)))
            ax1.set_yticks(np.arange(len(self.input)))
            fig.colorbar(im, ax=ax1)
            fig.savefig(f'./graphs/confusion_{name}', dpi=300);


In [ ]:
learning_rate = 0.1
epochs = 10000
nn_A = NeuralNetwork(2, 2, 1)
nn_A.train(x, y, learning_rate, epochs)
nn_A.predict(x)

### Question B

Load raw data

In [ ]:
data = np.loadtxt("TP3-ej3-digitos.txt", dtype=np.uint8) 
plt.imshow(data);

Flatten data

In [ ]:
data_flattened = data.flatten()
data_flattened = np.reshape(data_flattened, (10, 35))

print(data_flattened.shape)
data_flattened

Initialize x input (flattened digit) and y output (result of whether digit is pair)

In [ ]:
x = data_flattened
y = np.array([1, 0, 1, 0, 1, 0, 1, 0, 1, 0]).reshape(-1, 1)

plt.figure(figsize=(10, 3))
for i, img in enumerate(data_flattened):
    plt.subplot(1, 10, i+1)
    plt.title('Label:' + str(y[i]))
    plt.axis('off')
    plt.imshow(np.reshape(img, (7,5)))

In [ ]:
input_size = 35
hidden_size = 10
output_size = 1

learning_rate = 0.001
epochs = 10000

nn_B = NeuralNetwork(input_size, hidden_size, output_size)
nn_B.train(x, y, learning_rate, epochs)
nn_B.predict(x)

In [ ]:
digit = 4

plt.figure(figsize=(2,2))
plt.imshow(np.reshape(data_flattened[digit], (7,5)));
res = nn_B.predict(data_flattened[digit])
print('digit is even:', res)

In [ ]:
threshold = 0.5
res = nn_B.predict(data_flattened[digit])
is_even = res > threshold
print(f'The digit is even: {is_even}')

### Exercise C

Initialize data
- x: 10*35 (10 digits * 35 pixels each)
- y: 10*1 (can be classified to 10 possible digits)

In [ ]:
x = data_flattened
y = np.arange(10).reshape(-1, 1)


plt.figure(figsize=(10, 3))
for i, img in enumerate(data_flattened):
    plt.subplot(1, 10, i+1)
    plt.title('Label:' + str(y[i]))
    plt.axis('off')
    plt.imshow(np.reshape(img, (7,5)))

In [ ]:
input_size = 35
hidden_size = 10
output_size = 10

learning_rate = 0.00015
epochs = 100000

nn_C = NeuralNetwork(input_size, hidden_size, output_size, one_hot=True, activation_function='reLU')
nn_C.train(x, y, learning_rate, epochs)
nn_C.predict(x)

In [ ]:
digit = 3

plt.figure(figsize=(2,2))
plt.axis('off')
plt.imshow(np.reshape(data_flattened[digit], (7,5)));
res = nn_C.predict(data_flattened[digit])
print(res)
print('digit is:', np.argmax(res))

In [ ]:
def salt_pepper_noise(img, salt_probability, pepper_probability):
    img_copy = np.copy(img)
    salt = np.random.rand(img.size) < salt_probability
    pepper = np.random.rand(img.size) < pepper_probability
    img_copy[salt] = 1
    img_copy[pepper] = 0
    return img_copy

def random_noise(img):
    img_copy = np.copy(img)
    img_copy = img_copy + np.random.rand(img_copy.size)
    return img_copy

def gaussian_noise(img, mean, std):
    img_copy = np.copy(img)
    img_copy = img_copy + np.random.normal(mean, std, img_copy.size)
    return img_copy

In [ ]:
salt_pepper_digit = salt_pepper_noise(data_flattened[digit], 0.05, 0.05)
random_digit = random_noise(data_flattened[digit])
gauss_digit = gaussian_noise(data_flattened[digit], 0.1, 0.1)

noised_data = [salt_pepper_digit, random_digit, gauss_digit]
titles = ['S&P', 'Random', 'Gauss']

plt.figure(figsize=(4,3))
for i, img in enumerate(noised_data):
    plt.subplot(1, 3, i+1)
    plt.title(titles[i])
    plt.axis('off')
    plt.imshow(np.reshape(img, (7,5)));

In [ ]:
np.argmax(nn_C.predict(random_digit))

In [ ]:
#analyzing

In [ ]:
#analyzing

In [ ]:
nn_A.graph('loss_epoch')
nn_A.graph(graph_type='accuracy_epoch')

In [ ]:
nn_B.graph('loss_epoch')
nn_B.graph(graph_type='accuracy_epoch')

In [ ]:
nn_C.graph('loss_epoch')

In [ ]:
nn_C.graph(graph_type='confusion')